In [40]:
import csv
from math import exp
from numpy import *
from scipy.optimize import fmin, fmin_cg, fmin_powell, fmin_bfgs
    
# Prepare the data
    
def getIiter(ifname):
    """
    Get the iterator from a csv file with filename ifname
    """
    ifile = open(ifname, 'r')
    iiter = csv.reader(ifile)
    iiter.__next__()
    return iiter

def parseRow(s):
    y = [int(x) for x in s]
    lab = y[0]
    z = y[1:]
    return (lab, z)

def getAllRows(ifname):
    iiter = getIiter(ifname)
    x = []
    l = []
    for row in iiter:
        lab, z = parseRow(row)
        x.append(z)
        l.append(lab)
    return x, l

def cutData(x, y):
    """
    70% training
    30% testing
    """
    m = len(x)
    t = int(m * .7)
    return [(x[:t], y[:t]), (x[t:], y[t:])]

def num2IndMat(l):
    t = array(l)
    tt = [vectorize(int)((t == i)) for i in range(10)]
    return array(tt).T

def readData(ifname):
    x, l = getAllRows(ifname)
    t = [[1] + y for y in x]
    #return array(t), num2IndMat(l)
    return array(t), array(l)
    
#x, y = readData('train.csv')
#[(trainX, trainY), (testX, testY)] = cutData(x, y)
X, y = readData('train.csv')
#m = len(X)
#n = len(X[0]) - 1
#m = len(trainX)
#n = len(trainX[0]) - 1

In [34]:
def sigmoid(z):
# Computes the Sigmoid function of z
    g = 1.0 / (1.0 + e ** (-1.0 * z))

    return g 
    
def compute_cost(theta,X,y,lambd): 
# Computes cost function
    theta=theta.reshape(len(theta),1)
    h = sigmoid(X.dot(theta))
    m = len(y)
    h = h.reshape(len(h),1)
    y = y.reshape(len(y),1)
    theta[0] = 0
    
    #J = (1.0/m)*sum(-y*(log(h)) - (1-y)*(log(1-h))) + (lambd/(2*m))*sum(theta.T*theta.T)
    J = (1.0/m)*sum(-y*(log(h)) - (1-y)*(log(1-h)))

    return  J
    
def compute_grad(theta, X, y, lambd):
# Computes gradient of cost function 
    theta = theta.reshape(len(theta),1)
    h = sigmoid(X.dot(theta))
    h = h.reshape(len(h),1)
    y = y.reshape(len(y),1)
    m = len(y)
    theta[0] = 0
    
    #grad = (1.0/m)*(X.T.dot((h-y)).T) + (lambd/m)*theta.T
    grad = (1.0/m)*(X.T.dot((h-y)).T) 
    grad = ndarray.flatten(grad)

    return grad

def prediction(theta, X):
# Predicts the number [0:9] based on learned logistic regression data
    a, b = X.shape
    pred = zeros(shape=(a, 1))
    h = sigmoid(X.dot(theta.T))
    
    pred=argmax(h, axis=1)  
    
    return pred

def oneVsAll(X, y, num_labels, lambd):
    
    m, n = X.shape
    all_theta = zeros(shape=(num_labels, n))
    initial_theta = zeros(shape=(n, 1))
    y_t = ones(shape=(len(y),num_labels))
    y = y.reshape(len(y))
    
    for i in range(0, num_labels):
        y_t[:,i] = y
        y_t[y != i, i] = 0
        y_t[y == i, i] = 1
    
    for i in range(0, num_labels):
        #optimal_theta = op.fmin_cg(f = compute_cost, x0 = initial_theta, fprime = compute_grad, args = (X, y_t[:,i], lambd), maxiter=300)
        #optimal_theta = op.fmin_cg(f = compute_cost, x0 = initial_theta, fprime = compute_grad, args = (X, y_t[:,i], lambd), maxiter=20)
        optimal_theta = fmin_cg(f = compute_cost, x0 = initial_theta, fprime = compute_grad, args = (X, y_t[:,i], lambd), maxiter=20)
        #optimal_theta = op.fmin_cg(f = compute_cost, x0 = initial_theta, args = (X, y_t[:,i], lambd), maxiter=0)
        #optimal_theta = op.fmin(func = compute_cost, x0 = initial_theta, args = (X, y_t[:,i], lambd), maxiter=0)
        #Result = op.minimize(fun = compute_cost, x0 = initial_theta, args = (X, y_t[:,i],lambd), jac = compute_grad, method = 'TNC', options={'maxiter':20})
        #Result = op.minimize(fun = compute_cost, x0 = initial_theta, args = (X, y_t[:,i],lambd), jac = compute_grad, method = 'TNC', options={'maxiter':20})
        #optimal_theta = Result.x;
        all_theta[i,:] = optimal_theta
    return all_theta

In [42]:
shape(trainX)

(29399, 785)

In [46]:
trainX = multiply(trainX, 1/255)

In [47]:
compute_grad(zeros((785,)), trainX, trainY[:,0], 0)

array([  1.57808489e-03,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   8.40364718e-06,   3.13469379e-05,
         1.44062523e-05,   6.00260513e-07,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   5.80251829e-06,
         3.12802423e-05,   1.22052971e-04,   1.57268254e-04,
         2.69583666e-04,   3.37546495e-04,   3.54487181e-04,
         3.30476760e-04,   3.73962300e-04,   3.42481971e-04,
         3.06332948e-04,   3.03731820e-04,   2.08423789e-04,
         1.37326266e-04,

In [48]:
print(trainX, trainY[:,0])

[[ 0.00392157  0.          0.         ...,  0.          0.          0.        ]
 [ 0.00392157  0.          0.         ...,  0.          0.          0.        ]
 [ 0.00392157  0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.00392157  0.          0.         ...,  0.          0.          0.        ]
 [ 0.00392157  0.          0.         ...,  0.          0.          0.        ]
 [ 0.00392157  0.          0.         ...,  0.          0.          0.        ]] [0 1 0 ..., 0 1 0]


In [17]:
num_labels = 10
X, y = readData('train.csv')
m = len(X)
n = len(X[0]) - 1
lambd = 0

In [28]:
X = multiply(X, 1 / 255)

In [41]:
[(X, y), (t, tt)] = cutData(X, y)
print(shape(X), shape(y))

(29399, 785) (29399,)


In [35]:
all_theta = oneVsAll(X, y, num_labels, lambd)
print(all_theta)

         Current function value: 0.025578
         Iterations: 20
         Function evaluations: 55
         Gradient evaluations: 55
         Current function value: 0.025573
         Iterations: 20
         Function evaluations: 56
         Gradient evaluations: 56
         Current function value: 0.076762
         Iterations: 20
         Function evaluations: 47
         Gradient evaluations: 47
         Current function value: 0.097893
         Iterations: 20
         Function evaluations: 48
         Gradient evaluations: 48
         Current function value: 0.054952
         Iterations: 20
         Function evaluations: 52
         Gradient evaluations: 52
         Current function value: 0.096211
         Iterations: 20
         Function evaluations: 45
         Gradient evaluations: 45
         Current function value: 0.042095
         Iterations: 20
         Function evaluations: 58
         Gradient evaluations: 58
         Current function value: 0.050826
         Iterations:

In [37]:
p=prediction(all_theta[:,:],X)

t = (p == y.T)

print ("Train Accuracy is: ", mean(t)*100)

Train Accuracy is:  91.2003809653


In [2]:
#Calculate the cost function
    
def sigmoid(x):
    return 1 / (1 + exp(-x))
    
vSigmoid = vectorize(sigmoid)
vLog = vectorize(log)

def costFunction(theta, x, y):
    sigxt = vSigmoid(dot(x, theta))
    cm = (- y * vLog(sigxt) - (1 - y) * vLog(1 - sigxt)) / m / N
    return sum(cm)

def unflatten(flatTheta):
    return [flatTheta[i * N : (i + 1) * N] for i in range(n + 1)]

def costFunctionFlatTheta(flatTheta):
    return costFunction(unflatten(flatTheta), trainX, trainY)

def costFunctionFlatTheta1(flatTheta):
    return costFunction(flatTheta.reshape(785, 10), trainX, trainY)

N = len(trainY[0])

initTheta = zeros(((n + 1), N))
flatInitTheta = ndarray.flatten(initTheta)
flatInitTheta1 = initTheta.reshape(1, -1)